# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [28]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [29]:
# Import cities file as DataFrame
cities_pd = pd.read_csv("output/cities.csv")
cities_pd.head()



,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Lashio,22.9333,97.7500,66.45,98,30,0.47,MM,1652566176
1,1,Ixtapa,20.7000,-105.2000,82.22,69,75,6.91,MX,1652566612
2,2,São João da Barra,-21.6403,-41.0511,72.86,72,98,8.93,BR,1652566171
3,3,Avarua,-21.2078,-159.7750,75.25,88,75,5.75,CK,1652566613
4,4,Grand Gaube,-20.0064,57.6608,75.60,77,5,3.00,MU,1652566614


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [30]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [31]:
# Store 'Lat' and 'Lng' into  locations 
locations = cities_pd[["Lat", "Lng"]].astype(float)

humidity = cities_pd["Humidity"].astype(float)

In [32]:
# Create a  Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights= humidity, 
                                 dissipating=False, max_intensity=50,
                                 point_radius = 1)

fig.add_layer(heat_layer)


fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [33]:
# Show new df with max temp between 70 and 80 degrees, wind speed less than 10mph, and zero clouds
ideal_temp_df = cities_pd.loc[(cities_pd["Max Temp"] > 70) & (cities_pd['Max Temp'] < 80) & (cities_pd['Wind Speed']<10) & (cities_pd['Cloudiness']==0)]
ideal_temp_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
5,5,Arraial do Cabo,-22.9661,-42.0278,73.26,73,0,9.22,BR,1652566614
190,190,Baghdad,33.3406,44.4009,73.31,21,0,9.22,IQ,1652566589
248,248,Mossamedes,-15.1961,12.1522,71.44,68,0,1.66,AO,1652565894
301,301,Khuzdar,27.7384,66.6434,77.47,20,0,7.23,PK,1652566759
349,349,Buraidah,26.3260,43.9750,77.45,11,0,9.46,SA,1652566783
551,551,Sabha,27.0377,14.4283,70.61,25,0,9.91,LY,1652566895


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [34]:
hotel_df = ideal_temp_df
hotel_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
5,5,Arraial do Cabo,-22.9661,-42.0278,73.26,73,0,9.22,BR,1652566614
190,190,Baghdad,33.3406,44.4009,73.31,21,0,9.22,IQ,1652566589
248,248,Mossamedes,-15.1961,12.1522,71.44,68,0,1.66,AO,1652565894
301,301,Khuzdar,27.7384,66.6434,77.47,20,0,7.23,PK,1652566759
349,349,Buraidah,26.3260,43.9750,77.45,11,0,9.46,SA,1652566783
551,551,Sabha,27.0377,14.4283,70.61,25,0,9.91,LY,1652566895


In [35]:
hotel_df["Hotel Name"] = ""
hotel_df

c:\Users\jlper\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
5,5,Arraial do Cabo,-22.9661,-42.0278,73.26,73,0,9.22,BR,1652566614,
190,190,Baghdad,33.3406,44.4009,73.31,21,0,9.22,IQ,1652566589,
248,248,Mossamedes,-15.1961,12.1522,71.44,68,0,1.66,AO,1652565894,
301,301,Khuzdar,27.7384,66.6434,77.47,20,0,7.23,PK,1652566759,
349,349,Buraidah,26.3260,43.9750,77.45,11,0,9.46,SA,1652566783,
551,551,Sabha,27.0377,14.4283,70.61,25,0,9.91,LY,1652566895,


In [36]:
# params dictionary to update each iteration
params = {
    "radius": 50000,
    "types": "lodging",
    "key": g_key
}

# Use the lat/lng we recovered to identify airports
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    hotel = requests.get(base_url, params=params)
    
#     print the name_address url, avoid doing for public github repos in order to avoid exposing key
#     print(name_address.url)

    # convert to json
    hotel = hotel.json()
    # print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel["results"][0]["name"]
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

c:\Users\jlper\anaconda3\envs\PythonData\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [37]:
hotel_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
5,5,Arraial do Cabo,-22.9661,-42.0278,73.26,73,0,9.22,BR,1652566614,Pousada Porto Praia
190,190,Baghdad,33.3406,44.4009,73.31,21,0,9.22,IQ,1652566589,Baghdad Hotel
248,248,Mossamedes,-15.1961,12.1522,71.44,68,0,1.66,AO,1652565894,Hotel Chik Chik Namibe
301,301,Khuzdar,27.7384,66.6434,77.47,20,0,7.23,PK,1652566759,Hostel 6
349,349,Buraidah,26.3260,43.9750,77.45,11,0,9.46,SA,1652566783,Mövenpick Qassim
551,551,Sabha,27.0377,14.4283,70.61,25,0,9.91,LY,1652566895,Foundation Taaj Asala for Historical Research ...


In [38]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [39]:
# Add marker layer ontop of heat map
fig = gmaps.figure()

locations = hotel_df[["Lat", "Lng"]].astype(float)

humidity = hotel_df["Humidity"].astype(float)

heat_layer = gmaps.heatmap_layer(locations, weights= humidity, 
                                 dissipating=False, max_intensity=50,
                                 point_radius = 1)

fig.add_layer(heat_layer)

coordinates = locations
markers = gmaps.marker_layer(coordinates)     # marker_layer is the red markers on map, coordinates are the ones we give

# Add the layer to the map
fig.add_layer(markers)

fig

Figure(layout=FigureLayout(height='420px'))